In [71]:
#from build_sentence_corp import *
#from prepare_input import *

In [2]:
from model import *
import os

In [3]:
import pickle
f = open("./data/T_matrix.pickle",'rb')
_T = pickle.load(f)
f.close()

In [32]:
embedding_vector_size = 300

tf.reset_default_graph()
training_inputs = tf.placeholder(shape=[None, 300, 100], dtype=tf.float32)  

In [33]:
k = 14
g = tf.Graph()
#with g.as_default():
T = tf.transpose(tf.cast(tf.get_variable("T", initializer=_T), dtype=tf.float32))
M = tf.get_variable("M", [embedding_vector_size, embedding_vector_size], dtype=tf.float32)

In [34]:
zs = attention(training_inputs, M)
rs = reconstruct_embed(k, zs, training_inputs, M, T)

In [38]:
embedding_file_path = './data/batch_embedding/'
sent_embed_files = os.listdir(embedding_file_path)[:300]
batch_size = 50
max_val = batch_size-1
n = 20
loss = total_loss(rs, zs, T, training_inputs, max_val, n)

## Adam Optimizer

In [39]:
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

## Tensorflow Seesion

In [43]:
variables_names = [v.name for v in tf.trainable_variables()]
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)    
    for i in range(5):
        print("Epoch: ", i)
        for files in sent_embed_files:
            #print("file: ", files)
            filename = embedding_file_path + files
            with open(filename, 'rb') as f:
                sent_embedding_sample = pickle.load(f)                                        
            ls, _, Tk = sess.run([loss, train_op, T], feed_dict={training_inputs:sent_embedding_sample})
        print("loss", ls)


Variable:  T:0
Shape:  (14, 300)
Variable:  M:0
Shape:  (300, 300)
Variable:  dense/kernel:0
Shape:  (300, 14)
Variable:  dense/bias:0
Shape:  (14,)
Epoch:  0
loss 78.285126
Epoch:  1
loss 78.28513
Epoch:  2
loss 78.28513
Epoch:  3
loss 78.28513
Epoch:  4
loss 78.285126


In [49]:
import gensim
embedding_vector_size = 300
embedding_path = 'data/GoogleNews-vectors-negative300.bin'
embedding = gensim.models.KeyedVectors.load_word2vec_format(embedding_path, 
                             binary=True)


In [69]:
embedding.most_similar(positive=[Tk.T[9]], topn=10)

[('potato_leek_soup', 0.7608047723770142),
 ('fried_onion_rings', 0.7577475309371948),
 ('homemade_peach_cobbler', 0.7464830875396729),
 ('crispy_fries', 0.7408971786499023),
 ('spinach_lasagna', 0.7329265475273132),
 ('shoestring_fries', 0.7310258746147156),
 ('spiced_nuts', 0.7295354604721069),
 ('bruschetta_topping', 0.7291461229324341),
 ('pickled_cucumbers', 0.7280831933021545),
 ('dessert', 0.7270943522453308)]